In [ ]:
#Document loading


In [76]:
#Chunking
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = WikipediaLoader(query="New York economy", load_max_docs=5).load() 

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=25)

split_documents = text_splitter.split_documents(docs)

print(len(split_documents)) 

for split in split_documents:
    print(split)

56
page_content='The economy of the State of New York is reflected in its gross state product in 2022 of $2.053 trillion, ranking third in size behind the larger states of California and Texas. If New York State were an independent nation, it would rank as the 10th largest economy in the world. However, in 2019, the multi-state, New York City-centered metropolitan statistical area produced a gross metropolitan product (GMP) of $US2.0 trillion, ranking first nationally by a wide margin and would also rank as the' metadata={'title': 'Economy of New York (state)', 'summary': "The economy of the State of New York is reflected in its gross state product in 2022 of $2.053 trillion, ranking third in size behind the larger states of California and Texas. If New York State were an independent nation, it would rank as the 10th largest economy in the world. However, in 2019, the multi-state, New York City-centered metropolitan statistical area produced a gross metropolitan product (GMP) of $US2.0

In [ ]:
chroma_document_store.delete_collection()

In [77]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
os.environ['OPENAI_API_KEY'] = ''
  
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small") 

chroma_document_store = Chroma( 
    collection_name="chapter7db_documents_store", 
    embedding_function=embeddings_model, 
    persist_directory="chapter7db",
)

chroma_document_store.add_documents(split_documents)
chroma_document_store.persist()

print("There are", chroma_document_store._collection.count(), "in the collection")




There are 56 in the collection


In [34]:

similar_documents = chroma_document_store.similarity_search(query="what was the unemployment rate in nyc")
#what was the gross state product in 2022
#where do the largest companies have their offices
#The borough contained over 500 million square feet

print(similar_documents)


[Document(page_content="York City's unemployment rate fell to its record", metadata={'source': 'https://en.wikipedia.org/wiki/Economy_of_New_York_City', 'summary': 'The economy of New York City encompasses the largest municipal and regional economy in the United States. In 2022, the New York metropolitan area generated a gross metropolitan product (GMP) of US$2.1 trillion, with a population of 23.6 million people.  Anchored by Wall Street in Lower Manhattan, New York City has been characterized as the world\'s premier financial center. The city is home to the New York Stock Exchange (NYSE) and Nasdaq, the world\'s two largest stock exchanges by both market capitalization and trading activity.\nNew York City, anchored by Manhattan, is the world\'s leading center of banking, finance, and communication. It is home to the NYSE on Wall Street. Many of the world\'s largest corporations are headquartered in Manhattan. The borough contained over 500 million square feet (46.5 million m2) of off

In [78]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

os.environ['LANGCHAIN_API_KEY'] = 'ls__950b2ad6e3e34afea6c9ba3461c5369a'
os.environ['LANGCHAIN_PROJECT'] = "book-chapter-7" 
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"

llm = ChatOpenAI(model="gpt-4", temperature=0.0)
retriever = chroma_document_store.as_retriever(search_kwargs={"k": 20})

template = """
You are an assistant specializing in question-answering tasks based on provided document excerpts.
Your task is to analyze the given excerpts and formulate a final answer, citing the excerpts as "SOURCES" for reference. 
If the answer is not available in the excerpts, clearly state "I don't know." 
Ensure every response includes a "SOURCES" section, even if no direct answer can be provided.

QUESTION: {question}
====================================
EXCERPTS:
{chroma_documents}
====================================
FINAL ANSWER (with SOURCES if applicable):
"""
prompt = ChatPromptTemplate.from_template(template)


def format_chroma_docs(chroma_docs) -> str:
    return "\n".join(
        f"Content: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in chroma_docs
    )

rag_chain_from_docs = (
    {"chroma_documents": retriever | format_chroma_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "what stock exchanges are located in the big apple"
rag_chain_from_docs.invoke(question)



'The stock exchanges located in New York City, often referred to as the "Big Apple", are the New York Stock Exchange and Nasdaq. These are the world\'s two largest stock exchanges by market capitalization of their listed companies. \n\nSOURCES: \n- https://en.wikipedia.org/wiki/New_York_City\n- https://en.wikipedia.org/wiki/Economy_of_New_York_City'

In [ ]:
#Chunking
import os
from datetime import datetime
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
import chromadb
from langchain.vectorstores import Chroma
from langchain.docstore import InMemoryDocstore
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter
os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ[' LANGCHAIN_PROJECT'] = "book-chapter-6" 

llm = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-1106-preview")
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

chroma_client = chromadb.PersistentClient(path="./embeddings")

history_vectorstore = Chroma(collection_name="conversation_history", client=chroma_client, embedding_function=embeddings)

retriever = history_vectorstore.as_retriever(search_kwargs=dict(k=2))

similar_documents = history_vectorstore.similarity_search(query="do i need a torch", k=1)

print(history_vectorstore._collection.get())  #without embeddings
print(history_vectorstore._collection.get(include=['embeddings'])) # with embeddings

document_count = history_vectorstore._collection.count()
print(document_count)
for document in similar_documents:
    print(document)
